In [6]:
from FAdo.fa import *
from FAdo.reex import *
from FAdo.fio import *
from FAdo.rndadfa import *

import lark

In [10]:
class CCount(Unary):
	def __init__(self, arg, min, max = None, sigma=None):
		self.arg = arg
		self.min = min
		self.max = "inf" if max == -1 else max
		self.sigma = sigma

	def __repr__(self):
		"""Representation of the regular expression's syntactical tree."""
		return 'CCount({},[{}])'.format(self.arg, self.min if not self.max else '{},{}'.format(self.min, self.max))
	
	def linearForm(self): # https://www.dcc.fc.up.pt/~nam/resources/publica/51480046.pdf
		return self.arg.linearForm()

	def derivative(self, sigma): # how ...?
		# add power attribute to CCount class and keep count?
		# do i even need this? cant eval a word if i dont have it....
		# if self.max == "inf" or self.max == None:
		# 	if self.min == 0:
		# 		return CEmptySet()
		# 	elif self.min == 1:
		# 		return CEpsilon()
		# 	else:
		# 		d = self.arg.derivative(sigma)
		# 		if d == CEpsilon():
		# 			return CCount(self.arg, self.min, self.max, sigma)
		# 		else:
		# 			return d, CCount(self.arg, self.min, self.max, sigma)
		# else:
		# 	if self.min == self.max:
		# 		return CEpsilon()
		# 	else:
		# 		return CCount(self.arg, self.min, self.max-1, sigma), self.arg
		pass

In [ ]:
class BuildRegexpCC(BuildRegexp):
    def pow_min(self, s, inf=False):
        (arg, n_r) = s
        n = n_r.children[0].value
        if inf:
            r = CCount(arg, n, -1, self.sigma)
        else:
            r = CCount(arg, n, self.sigma)
        return r

    def pow_minmax(self, s):
        (arg, n_mi, n_ma) = s
        n_min = n_mi.children[0].value
        n_max = n_ma.children[0].value
        r = CCount(arg, n_min, n_max, self.sigma)
        return r
    
    def pow_inf(self, s):
        return self.pow_min(s, True)
    
regGrammar = lark.Lark.open("regexp_test.lark", start="rege", parser="lalr")

# available:
# b^[1] -> 'b'
# b^[1,2] -> 1 <= 'b' <= 2
# b^[1,...] -> 1 <= 'b' <= inf

tree = regGrammar.parse("(b^[1,3])")
reg = BuildRegexpCC(context={"sigma": None}).transform(tree)
reg

CConcat(CAtom(a),CCount(b,[1,3]))